In [24]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [25]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\aslig\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [26]:
mars_facts = {}

RED_PLANET_URL       = 'https://redplanetscience.com/'
MARS_IMAGES_URL      = 'https://spaceimages-mars.com/'
PLANET_FACTS_URL     = 'https://galaxyfacts-mars.com/'
MARS_HEMISPHERES_URL = 'https://marshemispheres.com/'

In [27]:
browser.visit(RED_PLANET_URL)
time.sleep(1)

In [28]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_items = soup.find_all('div', class_='list_text')

print(type(news_items))

<class 'bs4.element.ResultSet'>


In [29]:
news_title = None
news_body  = None
featured_news = []

for news_item in news_items:
    
    news_title = news_item.find('div',class_='content_title').text
    news_body = news_item.find('div',class_="article_teaser_body").text
    
    featured_news.append({"news_title"     : news_title, "news_paragraph" : news_body}) 
    
    mars_facts["featured_news"]=featured_news
    
    break #only take first news item

In [30]:
print(mars_facts)

{'featured_news': [{'news_title': "A Year of Surprising Science From NASA's InSight Mars Mission", 'news_paragraph': "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."}]}


In [31]:
browser.visit(MARS_IMAGES_URL)
time.sleep(1)

In [32]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [33]:
mars_images_url = browser.url
mars_image_wrapper = soup.find_all('div', class_='floating_text_area')[0]
img_target_url = mars_image_wrapper.find('a')['href']
featured_image_url = [mars_images_url + img_target_url]

mars_facts["featured_image"] = featured_image_url    

In [34]:
print(mars_facts)

{'featured_news': [{'news_title': "A Year of Surprising Science From NASA's InSight Mars Mission", 'news_paragraph': "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."}], 'featured_image': ['https://spaceimages-mars.com/image/featured/mars3.jpg']}


In [35]:
browser.visit(PLANET_FACTS_URL)
time.sleep(1)


In [36]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [37]:
df_column_1_name = "Description"
df_column_2_name = "Mars"
df_column_3_name = "Earth"

descriptions = []
mars_fcts   = []
earth_fcts  = []

mars_idx  = 0
earth_idx = 1

planet_facts_wrapper = soup.find('div', class_='diagram')
planet_fact_table    = planet_facts_wrapper.find('table', class_="table")
planet_fact_rows     = planet_fact_table.find_all('tr')

for planet_fact_row in planet_fact_rows:
    row_desc = planet_fact_row.find('th').text.strip()
    descriptions.append(row_desc)
    
    facts = planet_fact_row.find_all('td')
    
    mars_fact  = facts[mars_idx].text.strip()
    earth_fact = facts[earth_idx].text.strip()
    
    mars_fcts.append(mars_fact)
    earth_fcts.append(earth_fact)

planet_facts_df = pd.DataFrame({
                                    df_column_1_name: descriptions, 
                                    df_column_2_name: mars_fcts,
                                    df_column_3_name: earth_fcts,
                               })
    
planet_facts_df.set_index(df_column_1_name,inplace=True)
planet_facts_df.head(7)

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [38]:
planet_facts_html = planet_facts_df.to_html( buf=None, 
                                             columns=None, 
                                             header=True, 
                                             index=True, 
                                             na_rep='NaN', 
                                             formatters=None, 
                                             float_format=None, 
                                             sparsify=None, 
                                             index_names=True, 
                                             justify=None, 
                                             bold_rows=True, 
                                             classes=None, 
                                             escape=True, 
                                             max_rows=None,
                                             max_cols=None, 
                                             show_dimensions=False, 
                                             notebook=False )

print(planet_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [39]:
mars_facts["mars_facts"] = [planet_facts_html]

In [40]:
print(mars_facts)

{'featured_news': [{'news_title': "A Year of Surprising Science From NASA's InSight Mars Mission", 'news_paragraph': "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet."}], 'featured_image': ['https://spaceimages-mars.com/image/featured/mars3.jpg'], 'mars_facts': ['<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance fr

In [41]:
browser.visit(MARS_HEMISPHERES_URL)
time.sleep(1)

In [42]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [43]:
hemisphere_image_urls = []

In [44]:
mars_hemispheres_wrapper = soup.find('div', class_='collapsible results')
hemispheres              = mars_hemispheres_wrapper.find_all('div', class_='item')
mars_hemispheres_url     = browser.url

for hemisphere in hemispheres:
    title               = hemisphere.find('h3').text.replace('Enhanced','')
    large_image_nav_url = mars_hemispheres_url + hemisphere.find('a')['href']

    browser.visit(large_image_nav_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mars_large_image_wrapper = soup.find('div', class_='downloads')
    image_container = mars_large_image_wrapper.find('ul')
    image_items = image_container.find_all('li')

    for image_item in image_items:
        image_pict_desc = image_item.find('a').text

        if 'Sample' == image_pict_desc:
            img_url = mars_hemispheres_url + image_item.find('a')['href']
            image_dict = {"title":title,"img_url":img_url}
            hemisphere_image_urls.append(image_dict)
            break

In [45]:
mars_facts["mars_images"] = hemisphere_image_urls

In [ ]:
print(mars_facts)

In [47]:
browser.quit()